In [89]:
import numpy as np
import pandas as pd
import datetime
from dateutil.parser import parse

In [90]:
import json

In [91]:
with open('snowfalls.json', 'r') as infile:
    snowfalls_json = json.load(infile)

# Explore the structure of the JSON-like object holding the snowfall amounts by month.

In [92]:
len(snowfalls_json) # one dictionary

1

In [93]:
type(snowfalls_json)

dict

In [94]:
snowfalls_json.keys() # the first-level key points to a list of dictionaries

dict_keys(['snowfalls'])

In [95]:
snowfalls_json['snowfalls'][0].keys() # the third level - keys of dicts inside the list

dict_keys(['date', 'data'])

In [96]:
snowfalls_json['snowfalls'][0]['date']

'201701'

In [97]:
snowfalls_json['snowfalls'][0]['data'].keys()

dict_keys(['description', 'data'])

In [98]:
snowfalls_json['snowfalls'][0]['data']['data'].keys() # the fourth level keys in 'data'

dict_keys(['USC00210018', 'USC00210075', 'USC00210146', 'USC00210157', 'US1MNAA0054', 'USC00210252', 'USC00210287', 'US1MNMW0003', 'USC00210355', 'USC00210387', 'US1MNCS0003', 'US1MNOT0003', 'US1MNSH0019', 'US1MNKD0001', 'USC00210643', 'USC00210667', 'USC00210746', 'US1MNSH0010', 'US1MNRV0008', 'US1MNRV0005', 'USC00210809', 'USC00210855', 'US1MNIS0008', 'USC00210939', 'US1MNCW0008', 'US1MNCW0003', 'USC00210974', 'USC00210981', 'USC00210989', 'US1MNKB0003', 'US1MNHN0134', 'USC00211063', 'USC00211065', 'USC00211074', 'USC00211174', 'USC00211198', 'US1MNHS0004', 'USC00211227', 'USC00211250', 'USC00211263', 'USC00211310', 'US1MNCN0001', 'US1MNCV0001', 'USC00211374', 'USC00211422', 'US1MNCV0027', 'USC00211448', 'USC00211468', 'US1MNFM0001', 'US1MNSL0015', 'US1MNDD0011', 'USC00211630', 'USC00211691', 'US1MNBW0008', 'US1MNSL0034', 'USC00211771', 'US1MNWG0033', 'USC00211840', 'US1MNPK0013', 'US1MNSL0096', 'USC00212023', 'USC00212038', 'US1MNOT0007', 'USC00212088', 'US1MNOT0015', 'USC00212142',

In [99]:
snowfalls_json['snowfalls'][0]['data']['data']['USC00210018'].keys()  # all stations for the first month's data - fifth level

dict_keys(['station_name', 'lat', 'lon', 'state', 'county', 'elev', 'date', 'values'])

In [100]:
snowfalls_json['snowfalls'][0]['data']['data']['USC00210018']['date']

'20170101'

In [102]:
parse('20170101')

datetime.datetime(2017, 1, 1, 0, 0)

In [103]:
snowfalls_json['snowfalls'][0]['data']['data']['USC00210018']['values'].keys() # 6 levels deep

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31'])

In [104]:
len(snowfalls_json['snowfalls']) # there are 11 months of data here

11

In [105]:
# Helper function to add the day key to the base date

def fix_dates(first_day_of_month, calendar_day):
    return first_day_of_month + datetime.timedelta(days=(int(calendar_day) - 1))

In [106]:
fix_dates(parse('20170101'), '15') # should be January 15, 2019

datetime.datetime(2017, 1, 15, 0, 0)

In [107]:
# snowfalls_json['snowfalls'][0]['data']['data']['USC00210018']['values']

In [108]:
def station_df(index, station):
    # We do not know if a value is 'M' for missing because there is 
    # less than 'T'race amounts of snow or simply not observed.  Therefore,
    # return np.nan (NaN)
    def convert_snow (amount):
        if (amount == 'T'):
            return 0.01
        elif (amount == 'M'):
            return np.nan
        else:
            return float(amount)
            
    
    first_day_of_month = parse(snowfalls_json['snowfalls'][index]['data']['data'][station]['date'])
    snowfall_dict = snowfalls_json['snowfalls'][index]['data']['data'][station]['values']
    
    dates = [fix_dates(first_day_of_month, date) for date in snowfall_dict.keys()]
    snow = [convert_snow(amount) for amount in snowfall_dict.values()]
    
    return pd.DataFrame({
        'station': station,
        'name': snowfalls_json['snowfalls'][index]['data']['data'][station]['station_name'],
        'county': snowfalls_json['snowfalls'][index]['data']['data'][station]['county'],
        'longitude': float( snowfalls_json['snowfalls'][index]['data']['data'][station]['lon'] ),
        'latitude': float( snowfalls_json['snowfalls'][index]['data']['data'][station]['lat'] ),
        'date': dates,
        'snowfall': snow
    })


# Compare the Stucture of the JSON object to the DataFrame we produce below
Both give all snowfall amounts for one repoting station for one month.  The DataFame is easier to work with but contains alot of repeated information.

In [109]:
snowfalls_json['snowfalls'][0]['data']['data']['USC00210018']

{'station_name': 'ADA',
 'lat': '47.30',
 'lon': '-96.52',
 'state': 'MN',
 'county': 'NORMAN',
 'elev': '907',
 'date': '20170101',
 'values': {'1': '0.0',
  '2': '2.0',
  '3': '4.2',
  '4': '0.0',
  '5': '0.0',
  '6': '0.0',
  '7': '0.0',
  '8': '0.0',
  '9': '0.5',
  '10': '1.0',
  '11': '3.5',
  '12': '0.0',
  '13': '0.0',
  '14': '0.0',
  '15': '0.0',
  '16': '0.0',
  '17': '0.0',
  '18': '0.0',
  '19': '0.0',
  '20': '0.0',
  '21': '0.0',
  '22': '0.0',
  '23': '0.0',
  '24': '0.0',
  '25': '0.0',
  '26': '0.0',
  '27': '0.0',
  '28': '0.0',
  '29': '0.0',
  '30': '0.0',
  '31': '0.0'}}

In [110]:
station_df(0,'USC00210018')

,station,name,county,longitude,latitude,date,snowfall
0,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-01,0.0
1,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-02,2.0
2,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-03,4.2
3,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-04,0.0
4,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-05,0.0
5,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-06,0.0
6,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-07,0.0
7,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-08,0.0
8,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-09,0.5
9,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-10,1.0


In [111]:
# Now grab all of the stations for one month - i.e., one position on the list:

def all_stations_one_month(index):
    
    stations = snowfalls_json['snowfalls'][index]['data']['data'].keys()
    
    list_of_df = [station_df(index, station) for station in stations]
    
    long_df = pd.concat(list_of_df)
    
    # Now - fix up the index:
    long_df.reset_index(drop=True, inplace=True)
    
    return long_df


In [112]:
date1 = all_stations_one_month(0)

In [113]:
date1.shape

(9641, 7)

In [114]:
date1.describe()

,longitude,latitude,snowfall
count,9641.000000,9641.000000,7289.000000
mean,-93.773023,45.621286,0.375334
std,1.423072,1.358493,1.018711
min,-96.870000,43.520000,0.000000
25%,-94.830000,44.660000,0.000000
50%,-93.500000,45.130000,0.000000
75%,-92.910000,46.810000,0.010000
max,-89.690000,48.840000,10.000000


In [115]:
date1

,station,name,county,longitude,latitude,date,snowfall
0,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-01,0.0
1,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-02,2.0
2,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-03,4.2
3,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-04,0.0
4,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-05,0.0
5,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-06,0.0
6,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-07,0.0
7,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-08,0.0
8,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-09,0.5
9,USC00210018,ADA,NORMAN,-96.52,47.3,2017-01-10,1.0


In [122]:
# Now piece this all together:
# Remember of json object is a global variable

def all_stations_all_months():
    
    number_of_stations = len(snowfalls_json['snowfalls'])
    
    # stations = snowfalls_json['snowfalls'][index]['data']['data'].keys()
    # remember we have one station with no data 201512 <Response [404]>
    # for the key '201512', the 'data' key maps to False
    
    list_of_df = [all_stations_one_month(index) for index in range(number_of_stations)\
                  if snowfalls_json['snowfalls'][index]['data']]
    
    long_df = pd.concat(list_of_df)
    
    # Now - fix up the index:
    long_df.reset_index(drop=True, inplace=True)
    
    return long_df


In [123]:
all_amounts_df = all_stations_all_months()

In [124]:
all_amounts_df.shape

(107058, 7)

In [126]:
all_amounts_df.describe()

,longitude,latitude,snowfall
count,107058.000000,107058.000000,78237.000000
mean,-93.746734,45.597274,0.498753
std,1.402567,1.339999,1.414293
min,-96.940000,43.520000,0.000000
25%,-94.610000,44.670000,0.000000
50%,-93.490000,45.140000,0.000000
75%,-92.890000,46.710000,0.010000
max,-89.690000,48.840000,25.000000


In [127]:
# Remove missing data
all_amounts_df.dropna(inplace=True)

In [128]:
all_amounts_df.shape

(78237, 7)

In [129]:
len(all_amounts_df['date'].unique())

301

In [130]:
len(all_amounts_df['station'].unique())

548

In [131]:
all_amounts_df.to_csv("snowfall.csv") # Save our work to this point

In [175]:
# While a map of snowfall for all 548 stations would be very detailed, this is more than what is 
# needed.  We will limit to the MSP metro counties

METROCOUNTIES = ['HENNEPIN', 'RAMSEY', 'ANOKA', 'CARVER', 'DAKOTA', 'SCOTT', 'WRIGHT', 'SHERBURNE', 'ISANTI',\
                'WASHINGTON', 'CHISAGO']

def filter_by_dates(df, start_date, end_date, counties=METROCOUNTIES):
    """For each station, return the cumulative snowfall from start_date to end_date, inclusive"""
    
    number_of_days = (end_date - start_date).days + 1 # since inclusive
    
    range_of_days = [start_date + datetime.timedelta(days=day) for day in range(number_of_days)]
    
    time_cond = df['date'].isin(range_of_days)
    
    county_cond = df['county'].isin(counties)
    
    combined_cond = time_cond & county_cond
    
    return df[combined_cond]


In [179]:
filtered_df = filter_by_dates(all_amounts_df, start_date, end_date)

In [178]:
def group_and_summarize_dataframe (df, grouping, stats):
    """Compute summary quantities for a pandas dataframe after grouping.
    
    Args:
        df: A pandas dataframe
        grouping: A column name or a list of column names to group
            the dataframe with.
        stats: A list of tuples. Each tuple is of the form:
            
            (name, column, function)
        
        where `name` is the desired output name for the summary quantity,
        `column` is the target column of the data frame, and `function` is 
        the function used to compute the summary.
        
    Returns:
        A pandas dataframe with the output summaries.
        
    Raises:
        None.  At this time, there is no error checking for the input parameters.
    """
        
    # Create an empty dataframe to store the results.
    results_df = pd.DataFrame()
    
    grouped_df = df.groupby(grouping)
    
    for name, column, agg_func in stats:
        results_df[name] = grouped_df[column].agg(agg_func)
        
    return results_df.reset_index()

In [180]:
snowtotal_df = group_and_summarize_dataframe(filtered_df, 'station',\
                                           [('name', 'name', lambda x: x.unique()),\
                                            ('county', 'county', lambda x: x.unique()),\
                                            ('longitude', 'longitude', lambda x: x.unique()),\
                                            ('latitude', 'latitude',  lambda x: x.unique()),\
                                            ('snowtotal', 'snowfall', 'sum')])

snowtotal_df

,station,name,county,longitude,latitude,snowtotal
0,US1MNAA0004,SAINT FRANCIS 4.0 E,ANOKA,-93.31,45.41,7.0
1,US1MNAA0054,ANOKA 1.3 SSE,ANOKA,-93.37,45.19,5.3
2,US1MNAA0056,BLAINE 2.2 NNW,ANOKA,-93.23,45.20,5.5
3,US1MNAA0068,EAST BETHEL 3.0 NE,ANOKA,-93.16,45.39,4.0
4,US1MNAA0073,ANOKA 6.5 N,ANOKA,-93.41,45.30,5.0
5,US1MNCG0001,RUSH CITY 2.8 NE,CHISAGO,-92.93,45.72,6.0
6,US1MNCG0003,CHISAGO CITY 1.8 ENE,CHISAGO,-92.88,45.36,4.5
7,US1MNCG0007,STACY 1.4 NW,CHISAGO,-93.02,45.40,5.5
8,US1MNCG0027,NORTH BRANCH 2.4 WSW,CHISAGO,-93.01,45.50,5.1
9,US1MNCG0028,STANCHFIELD 4.9 ESE,CHISAGO,-93.09,45.64,5.5


In [181]:
def metro_snowtotals_by_dates(df, start_date, end_date):
    
    filtered_df = filter_by_dates(df, start_date, end_date)
    
    total_df = group_and_summarize_dataframe(filtered_df, 'station',\
                                           [('name', 'name', lambda x: x.unique()),\
                                            ('county', 'county', lambda x: x.unique()),\
                                            ('longitude', 'longitude', lambda x: x.unique()),\
                                            ('latitude', 'latitude',  lambda x: x.unique()),\
                                            ('snowtotal', 'snowfall', 'sum')])
    return total_df


In [191]:
def lookup_snowtotals_by_emergency(snow_df, name):
    
    emergencies_df = pd.read_csv("episodes.csv")
    
    emergency_df = emergencies_df[emergencies_df.emergency == name]
    
    start_date = emergency_df.storm_begin_date.tolist()[0]
    end_date  = emergency_df.declaration_date.tolist()[0]
    
    start_date = parse(start_date)
    end_date = parse(end_date)
    
    snow_amounts_df = metro_snowtotals_by_dates(snow_df, start_date, end_date)
    
    return snow_amounts_df


In [193]:
lookup_snowtotals_by_emergency(all_amounts_df, 'Xerxes')

,station,name,county,longitude,latitude,snowtotal
0,US1MNAA0054,ANOKA 1.3 SSE,ANOKA,-93.37,45.19,3.8
1,US1MNAA0056,BLAINE 2.2 NNW,ANOKA,-93.23,45.20,3.4
2,US1MNCG0001,RUSH CITY 2.8 NE,CHISAGO,-92.93,45.72,4.0
3,US1MNCG0003,CHISAGO CITY 1.8 ENE,CHISAGO,-92.88,45.36,5.2
4,US1MNCG0007,STACY 1.4 NW,CHISAGO,-93.02,45.40,4.3
5,US1MNCG0016,NORTH BRANCH 1.3 NNW,CHISAGO,-92.97,45.53,4.2
6,US1MNCG0027,NORTH BRANCH 2.4 WSW,CHISAGO,-93.01,45.50,4.0
7,US1MNCV0001,CARVER 0.7 W,CARVER,-93.64,44.76,4.4
8,US1MNCV0008,WATERTOWN 0.5 NNW,CARVER,-93.85,44.97,4.0
9,US1MNCV0022,VICTORIA 1.6 WSW,CARVER,-93.69,44.86,4.4


In [194]:
# The code below was not mine - the source is given below
# https://geoffboeing.com/2015/10/exporting-python-data-geojson/

import geojson

def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [195]:
df_to_geojson(lookup_snowtotals_by_emergency(all_amounts_df, 'Xerxes'), ['station', 'name', 'snowtotal'])


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'station': 'US1MNAA0054',
    'name': 'ANOKA 1.3 SSE',
    'snowtotal': 3.8},
   'geometry': {'type': 'Point', 'coordinates': [-93.37, 45.19]}},
  {'type': 'Feature',
   'properties': {'station': 'US1MNAA0056',
    'name': 'BLAINE 2.2 NNW',
    'snowtotal': 3.4},
   'geometry': {'type': 'Point', 'coordinates': [-93.23, 45.2]}},
  {'type': 'Feature',
   'properties': {'station': 'US1MNCG0001',
    'name': 'RUSH CITY 2.8 NE',
    'snowtotal': 4.0},
   'geometry': {'type': 'Point', 'coordinates': [-92.93, 45.72]}},
  {'type': 'Feature',
   'properties': {'station': 'US1MNCG0003',
    'name': 'CHISAGO CITY 1.8 ENE',
    'snowtotal': 5.2},
   'geometry': {'type': 'Point', 'coordinates': [-92.88, 45.36]}},
  {'type': 'Feature',
   'properties': {'station': 'US1MNCG0007',
    'name': 'STACY 1.4 NW',
    'snowtotal': 4.3},
   'geometry': {'type': 'Point', 'coordinates': [-93.02, 45.4]}},
  {'type': 'Feature',
   

In [142]:
#  Finally - at the end --

def get_snowfall_geojson_by_emergency(snow_emergency):
    
    return df_to_geojson(lookup_snowtotals_by_emergency(all_amounts_df, snow_emegency),\
                         ['station', 'name', 'snowtotal'])


1